## Start by setting week number
#### Wait for ESPN data update until at least Wednesday morning, but run latest by Thursday night

https://www.espn.com/nfl/stats/team/_/stat/total/season/2022/seasontype/2

In [1]:
SET_WEEK = 12
week_name = 'week_' + str(SET_WEEK)
print(week_name)

week_12


In [2]:
from scrapers.scrape_2022 import get_2022_data
import pandas as pd
import numpy as np
import pickle

import warnings
warnings.filterwarnings('ignore')

### Models

In [3]:
logit = '../modeling/logit.pkl'
rf = '../modeling/rf.pkl'
gbm = '../modeling/gbm.pkl'
lgb = '../modeling/lgb.pkl'
catboost = '../modeling/catboost.pkl'

with open(logit, 'rb') as file:
    LOGIT = pickle.load(file)

with open(rf, 'rb') as file:
    RF = pickle.load(file)
    
with open(gbm, 'rb') as file:
    GBM = pickle.load(file)
    
with open(lgb, 'rb') as file:
    LGB = pickle.load(file)
    
with open(catboost, 'rb') as file:
    CATBOOST= pickle.load(file)

### 2022 data

In [4]:
%%time

X = get_2022_data()

# from boruta elimination method:
keep_cols = ['offense_points_per_game',
             'offense_downs_Fourth Downs_PCT',
             'offense_passing_AVG',
             'offense_passing_YDS/G',
             'offense_passing_RTG',
             'offense_receiving_AVG',
             'defense_points_per_game',
             'defense_passing_AVG',
             'defense_receiving_AVG',
             'defense_rushing_YDS/G',
             'defense_passing_SYL_per_game',
             'offense_downs_Third Downs_ATT_per_game',
             'defense_downs_Third Downs_ATT_per_game',
             'defense_downs_First Downs_penalty_ratio',
             'offense_passing_TD_per_game',
             'defense_rushing_TD_per_game',
             'defense_pass_TD_per_rush_TD',
             'offense_pass_TD_to_INT']

X = X[['Team', 'games_played', 'season'] + keep_cols].copy()

X.head(2)

Wall time: 20.5 s


,Team,games_played,season,offense_points_per_game,offense_downs_Fourth Downs_PCT,offense_passing_AVG,offense_passing_YDS/G,offense_passing_RTG,offense_receiving_AVG,defense_points_per_game,...,defense_receiving_AVG,defense_rushing_YDS/G,defense_passing_SYL_per_game,offense_downs_Third Downs_ATT_per_game,defense_downs_Third Downs_ATT_per_game,defense_downs_First Downs_penalty_ratio,offense_passing_TD_per_game,defense_rushing_TD_per_game,defense_pass_TD_per_rush_TD,offense_pass_TD_to_INT
0,Kansas City Chiefs,11,2022,29.6,75.0,8.1,315.2,104.6,12.3,22.1,...,10.4,102.0,22.0,11.727273,12.909091,0.051163,2.636364,0.545455,3.666667,3.625000
1,Buffalo Bills,11,2022,28.1,50.0,7.7,279.2,94.1,12.1,18.1,...,10.5,105.6,18.0,11.636364,12.727273,0.096070,2.090909,0.727273,1.750000,2.090909


In [5]:
print(X.shape)

(32, 21)


In [6]:
X.fillna(0, inplace = True)
X.replace([np.inf, -np.inf], 0, inplace=True)

In [7]:
X['games_played'].unique().tolist()

[11, 12]

In [8]:
# X[X['games_played'] == 8]

In [9]:
predictions = X[['Team', 'season']].copy()
predictions['week_marker'] = week_name

predictions['pred_logit'] = LOGIT.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_rf'] = RF.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_gbm'] = GBM.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_lgb'] = LGB.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_catboost'] = CATBOOST.predict_proba(X.drop(['Team', 'games_played', 'season'], 1))[:,1]
predictions['pred_mean'] = (predictions['pred_logit'] + predictions['pred_rf'] + predictions['pred_gbm'] + predictions['pred_lgb'] + predictions['pred_catboost']) / 5
predictions['pred_opt_combo'] = (predictions['pred_logit'] + 7*predictions['pred_catboost']) / 8

predictions.sort_values('pred_opt_combo', ascending = False, inplace = True)
predictions.head(3)

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_12,0.534502,0.333554,0.228499,0.332668,0.416116,0.369068,0.430914
1,Buffalo Bills,2022,week_12,0.548144,0.163632,0.099688,0.216142,0.370342,0.279590,0.392567
9,San Francisco 49ers,2022,week_12,0.278876,0.243749,0.130057,0.274180,0.363994,0.258171,0.353354


In [10]:
predictions.to_csv('weekly_runs_2022/' + week_name + '.csv', index = False)

### Check weekly run

In [11]:
predictions

,Team,season,week_marker,pred_logit,pred_rf,pred_gbm,pred_lgb,pred_catboost,pred_mean,pred_opt_combo
0,Kansas City Chiefs,2022,week_12,0.534502,0.333554,0.228499,0.332668,0.416116,0.369068,0.430914
1,Buffalo Bills,2022,week_12,0.548144,0.163632,0.099688,0.216142,0.370342,0.279590,0.392567
9,San Francisco 49ers,2022,week_12,0.278876,0.243749,0.130057,0.274180,0.363994,0.258171,0.353354
2,Miami Dolphins,2022,week_12,0.139067,0.270114,0.122227,0.268609,0.371979,0.234399,0.342865
3,Philadelphia Eagles,2022,week_12,0.187580,0.265670,0.123360,0.262715,0.355595,0.238984,0.334593
12,Dallas Cowboys,2022,week_12,0.216692,0.088931,0.074256,0.197687,0.347889,0.185091,0.331490
5,Cincinnati Bengals,2022,week_12,0.047255,0.131845,0.070544,0.217956,0.360020,0.165524,0.320925
11,Seattle Seahawks,2022,week_12,0.010193,0.065252,0.059173,0.196908,0.352513,0.136808,0.309723
20,New England Patriots,2022,week_12,0.050456,0.051460,0.075758,0.222721,0.340083,0.148096,0.303880
17,Tampa Bay Buccaneers,2022,week_12,0.028114,0.072168,0.073447,0.199974,0.342977,0.143336,0.303620
